In [3]:
import os, json
from dotenv import load_dotenv
from pymongo import MongoClient
from bson import json_util as bsonju

# --- setup ---
load_dotenv()
uri = os.getenv("MONGODB_URI")
db_name = os.getenv("MONGODB_DATABASE")
coll_name = "sales_invoices"
path = "data.json"

if not uri or not db_name:
    raise RuntimeError("Set MONGODB_URI and MONGODB_DATABASE in your .env")

# --- read data.json (Extended JSON preserved) ---
with open(path, "r", encoding="utf-8") as f:
    raw = f.read().strip()

data = bsonju.loads(raw)  # preserves {"$date": ...}

# accept either a single object or a single-element array
if isinstance(data, list):
    if len(data) != 1:
        raise ValueError("data.json must contain exactly one document")
    doc = data[0]
elif isinstance(data, dict):
    doc = data
else:
    raise TypeError("data.json must be a JSON object or a one-element array")

if "id" not in doc or not isinstance(doc["id"], str) or not doc["id"]:
    raise ValueError("Document must include a non-empty string 'id'")

# --- upload (upsert by 'id') ---
client = MongoClient(uri)
coll = client[db_name][coll_name]

res = coll.replace_one({"id": doc["id"]}, doc, upsert=True)

summary = {
    "acknowledged": res.acknowledged,
    "matched_count": res.matched_count,
    "modified_count": res.modified_count,
    "upserted_id": str(res.upserted_id) if res.upserted_id else None,
    "document_id": doc["id"],
    "items_count": len(doc.get("items") or []),
}

print(json.dumps(summary, ensure_ascii=False, indent=2, default=bsonju.default))


{
  "acknowledged": true,
  "matched_count": 0,
  "modified_count": 0,
  "upserted_id": "68f83665a268c4061366be55",
  "document_id": "8f6a2e4b-3d9a-4f2f-9c2a-7b1a0f4c9d31",
  "items_count": 6
}
